In [68]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('dataProcessed.csv')
print(df)

        location rooms  sqm_m2  price_normalized  trieu_met_vuong  \
0     Binh Chanh  1 PN      38             760.0            20.00   
1     Binh Chanh  2 PN      72            2780.0            38.61   
2     Binh Chanh  2 PN      72            2750.0            38.19   
3     Binh Chanh  2 PN      72            2249.0            31.24   
4     Binh Chanh  2 PN      70            2800.0            40.00   
...          ...   ...     ...               ...              ...   
1846     Thu Duc  2 PN      53            2399.0            45.70   
1847     Thu Duc  2 PN      52            1600.0            30.77   
1848     Thu Duc  2 PN      52             490.0             9.42   
1849     Thu Duc  2 PN      51            1700.0            33.33   
1850     Thu Duc  2 PN      57            1650.0            28.95   

      rooms_normalized  
0                    1  
1                    2  
2                    2  
3                    2  
4                    2  
...                ..

Dự đoán giá nhà theo khoảng
vì dữ liệu không có khoảng giá cụ thể nên sẽ tạo khoảng bằng các khoảng giá
dưới 5 tỷ = affordable apartmen
từ 5 tỷ đến 10 tỷ = mid-range apartment
trên 10 tỷ = luxury apartment
sau đó sẽ thêm 1 cột vào dữ liệu để có thể huấn luyện dữ liệu

KNN không thể làm việc trực tiếp với dữ liệu dạng chuỗi (string), nên cần chuyển đổi các giá trị này thành dạng số (integer) cho giá trị location và giá trị classify

In [63]:
# Tạo các nhóm dựa trên khoảng giá
bins = [100.0, 5000.0, 10000.0, float('inf')]
labels = ['affordable apartment', 'mid-range apartment', 'luxury apartment']

# Thêm cột "classify" vào DataFrame
df['classify'] = pd.cut(df['price_normalized'], bins=bins, labels=labels, right=False)

# Kiểm tra nếu cột 'classify' được thêm vào đúng cách
print(df[['price_normalized', 'classify']].head())

# Mã hóa cột 'classify' thành số nguyên theo thứ tự mong muốn
label_mapping = {'affordable apartment': 0, 'mid-range apartment': 1, 'luxury apartment': 2}
df['classify_encoded'] = df['classify'].map(label_mapping)

# Mã hóa cột 'location' thành số nguyên
label_encoder_location = LabelEncoder()
df['location_encoded'] = label_encoder_location.fit_transform(df['location'])

# Hiển thị DataFrame mới
print(df.head())

   price_normalized              classify
0             760.0  affordable apartment
1            2780.0  affordable apartment
2            2750.0  affordable apartment
3            2249.0  affordable apartment
4            2800.0  affordable apartment
     location rooms  sqm_m2  price_normalized  trieu_met_vuong  \
0  Binh Chanh  1 PN      38             760.0            20.00   
1  Binh Chanh  2 PN      72            2780.0            38.61   
2  Binh Chanh  2 PN      72            2750.0            38.19   
3  Binh Chanh  2 PN      72            2249.0            31.24   
4  Binh Chanh  2 PN      70            2800.0            40.00   

   rooms_normalized              classify classify_encoded  location_encoded  
0                 1  affordable apartment                0                 0  
1                 2  affordable apartment                0                 0  
2                 2  affordable apartment                0                 0  
3                 2  affordable apa

Tạo mô hình knn và huấn luyện chúng với k = 5
Ở đây để lấy được dự đoán được loại chung cư em cần biết:
    - location(vị trí của căn hộ)
    - rooms_normalized(số phòng)
    - sqm_m2(Diện tích căn chung cư)

In [64]:
# Tạo mô hình KNN với số lượng láng giềng (neighbors) mong muốn
k = 5
knn_model = KNeighborsClassifier(n_neighbors=k)

# Tách dữ liệu thành features và target
X = df[['location_encoded', 'rooms_normalized', 'sqm_m2']]  # Chỉ lấy các đặc trưng 'location_encoded', 'rooms_normalized' và 'sqm_m2'
y = df['classify_encoded']  # Sử dụng cột 'classify_encoded' làm target

# Huấn luyện mô hình
knn_model.fit(X, y)

KNeighborsClassifier()

Sau khi đã huấn luyện mô hình thành công, em sẽ dự đoán 1 căn hộ chung cư với
    vị trí: Bình Chánh
    số phòng: 2
    diện tích: 80

In [65]:
# Dự đoán classify của một căn hộ mới
new_data = pd.DataFrame({
    'location': ['Binh Chanh'],
    'rooms_normalized': [2],
    'sqm_m2': [80]
})

# Sử dụng label encoding cho vị trí của căn hộ mới
new_data['location_encoded'] = label_encoder_location.transform(new_data['location'])

# Chuẩn hóa dữ liệu mới
new_data_for_prediction = new_data[['location_encoded', 'rooms_normalized', 'sqm_m2']]

# Dự đoán classify của căn hộ mới
predicted_classify_encoded = knn_model.predict(new_data_for_prediction)

# Chuyển đổi kết quả dự đoán từ mã số phân loại sang nhãn
predicted_classify = [key for key, value in label_mapping.items() if value == predicted_classify_encoded[0]]
print("Predicted classify:", predicted_classify)

Predicted classify: ['affordable apartment']
